In [1]:
import numpy as np
import pandas as pd

### 将"长"透视为"宽"
在数据库和CSV中存储多时间序列的方式就是所谓的长格式或堆叠格式:

In [2]:
data = pd.read_csv('D:\\python\\np_pd_mat\\datasets\\macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [3]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
sub_data = data.reindex(columns=columns)
sub_data.index = periods.to_timestamp(freq='D', how='end').strftime('%Y-%m-%d')       # 转换为月末日期
sub_data[:4]

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6


In [4]:
ldata = sub_data.stack().reset_index().rename(columns={0: 'value'})
ldata[:4]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801


这种数据即所谓的多时间序列的长格式，或称为具有两个或更多个键的其他观测数据，表中的每一行表示一个时间点上的单个观测值，如果想要获取一个按date列时间戳索引的且每个不同的item独立一列的DataFrame：

In [5]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted[:4]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6


传递的前两个值是分别用作行和列索引的列，然后是可选的数值列以填充DataFrame，假设有两个数值列，想同时进行重塑：

In [6]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:4]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-1.440463
1,1959-03-31,infl,0.000,0.026542
2,1959-03-31,unemp,5.800,-0.780846
3,1959-06-30,realgdp,2778.801,-1.164856


如果遗漏最后一个参数，会得到一个含有多层列的DataFrame：

In [7]:
pivoted = ldata.pivot('date', 'item')
pivoted[:4]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.026542 -1.440463 -0.780846
1959-06-30  2.34  2778.801   5.1  0.752227 -1.164856 -0.721063
1959-09-30  2.74  2775.488   5.3 -0.195548  1.969107 -0.943325
1959-12-31  0.27  2785.204   5.6 -0.352669 -0.687990 -0.749889

In [8]:
pivoted['value'][:4]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6


注意pivot方法等价于使用set_index创建分层索引，然后调用unstack():

In [9]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:4]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.026542 -1.440463 -0.780846
1959-06-30  2.34  2778.801   5.1  0.752227 -1.164856 -0.721063
1959-09-30  2.74  2775.488   5.3 -0.195548  1.969107 -0.943325
1959-12-31  0.27  2785.204   5.6 -0.352669 -0.687990 -0.749889

### 将"宽"透视为"长"

In [10]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'], 'A': [1,2,3], 'B':[4,5,6],'C':[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [11]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [12]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [13]:
reshaped.reset_index()      # 使用reset_index将数据移回第一列

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


指定列的子集作为值列

In [14]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pd.melt的使用可以无须任何分组指标：

In [15]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
